In [1]:
import os
import gc
import sys
import numpy as np
from glob import glob
import xarray as xr

from subprocess import run, PIPE

from multiprocessing import Pool, cpu_count
from functools import partial

In [2]:
# Config
site = 'CLN'
lat, lon, elev = 40.5763, -111.6383, 2945

# Fixed to the length of ERA5 data cached locally
# Shorted from 1979 to start date of dataset as needed
start, end = 1998, 2019

In [4]:
isodir = '/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/era5/iso/'
sfcdir = '/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/era5/sfc/'
profdir = '/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/era5/profiles/'

In [5]:
isokeys = ['q', 't', 'u', 'v', 'vo', 'w', 'z', 'r']
sfckeys = ['100u', '100v', '10u', '10v', '2d', '2t', 'blh', 'cape', 'msl', 'sp']

In [17]:
yyyy = 2000
mm = 1

flist_raw = glob(isodir + '/%04d%02d/*'%(yyyy, mm))
flist = [f for f in flist_raw if f.split('.')[-5].split('_')[-1] in isokeys]

data_raw = xr.open_mfdataset(flist)
data_raw

<xarray.Dataset>
Dimensions:    (latitude: 81, level: 23, longitude: 113, time: 744)
Coordinates:
  * time       (time) datetime64[ns] 2000-01-01 ... 2000-01-31T23:00:00
  * latitude   (latitude) float64 50.0 49.75 49.5 49.25 ... 30.5 30.25 30.0
  * level      (level) float64 200.0 225.0 250.0 300.0 ... 950.0 975.0 1e+03
  * longitude  (longitude) float64 232.0 232.2 232.5 232.8 ... 259.5 259.8 260.0
Data variables:
    Q          (time, level, latitude, longitude) float32 dask.array<shape=(744, 23, 81, 113), chunksize=(24, 23, 81, 113)>
    utc_date   (time) int32 dask.array<shape=(744,), chunksize=(24,)>
    R          (time, level, latitude, longitude) float32 dask.array<shape=(744, 23, 81, 113), chunksize=(24, 23, 81, 113)>
    V          (time, level, latitude, longitude) float32 dask.array<shape=(744, 23, 81, 113), chunksize=(24, 23, 81, 113)>
    W          (time, level, latitude, longitude) float32 dask.array<shape=(744, 23, 81, 113), chunksize=(24, 23, 81, 113)>
    VO        

In [97]:
lats, lons = data_raw.latitude, data_raw.longitude - 360
lats, lons = np.meshgrid(lats, lons)

a = abs(lats - lat) + abs(lons - lon)
yi, xi = np.unravel_index(a.argmin(), a.shape)

data = data_raw.isel(longitude=yi, latitude=xi)
print('%f, %f'%(data.latitude.values, data.longitude.values-360))

40.500000, -111.750000
